In [1]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-otoxhn3t
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-otoxhn3t
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=411e27a3641f478e7f7ae50607476fedd97b4686b495f320c24ae1f3ee754eef
  Stored in directory: /tmp/pip-ephem-wheel-cache-sfyhp6lb/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [9]:
%%cu
#include <malloc.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

// ядро
__global__ void kernel(double* a, double* b, double* c, int n) {
    // получение текущей колонки
    int col = blockIdx.y*blockDim.y+threadIdx.y;
    // получение текущей строки
    int row = blockIdx.x*blockDim.x+threadIdx.x;
    double tmpSum = 0;
    // проверка, что не вышли за размерность
    if (row<n && col<n){
        for (int i = 0; i < n; i++) {
            tmpSum += a[row * n + i] * b[i * n + col];
        }
    }
    c[row * n + col] = tmpSum;
}


// функция перемножения матриц на GPU
double execute_gpu(double* a, double* b, double* c, int n, dim3 blocksPerGrid, dim3 threadsPerBlock){
    // количество байтов, которые нужно выделить для матрицы
    int n2b = n * n * sizeof(double);
     // Выделение памяти на устройстве
    double* adev = NULL;
    cudaError_t cuerr = cudaMalloc((void**)&adev, n2b);
    if (cuerr != cudaSuccess) {
        fprintf(stderr, "Cannot allocate device array for a: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }
    double* bdev = NULL;
    cuerr = cudaMalloc((void**)&bdev, n2b);
    if (cuerr != cudaSuccess) {
        fprintf(stderr, "Cannot allocate device array for b: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }
    double* cdev = NULL;
    cuerr = cudaMalloc((void**)&cdev, n2b);
    if (cuerr != cudaSuccess) {
        fprintf(stderr, "Cannot allocate device array for c: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }
    // Создание обработчиков событий
    cudaEvent_t start, stop;
    float gpuTime = 0.0f;
    cuerr = cudaEventCreate(&start);
    if (cuerr != cudaSuccess) {
        fprintf(stderr, "Cannot create CUDA start event: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }
    cuerr = cudaEventCreate(&stop);
    if (cuerr != cudaSuccess) {
        fprintf(stderr, "Cannot create CUDA end event: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }
    // Копирование данных с хоста на девайс
    cuerr = cudaMemcpy(adev, a, n2b, cudaMemcpyHostToDevice);
    if (cuerr != cudaSuccess) {
        fprintf(stderr, "Cannot copy a array from host to device: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }
    cuerr = cudaMemcpy(bdev, b, n2b, cudaMemcpyHostToDevice);
    if (cuerr != cudaSuccess) {
        fprintf(stderr, "Cannot copy b array from host to device: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }
    // Установка точки старта
    cuerr = cudaEventRecord(start, 0);
    if (cuerr != cudaSuccess) {
        fprintf(stderr, "Cannot record CUDA event: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }

    //Запуск ядра
    kernel <<< blocksPerGrid, threadsPerBlock >>> (adev, bdev, cdev, n);
    cuerr = cudaGetLastError();
    if (cuerr != cudaSuccess)
    {
        fprintf(stderr, "Cannot launch CUDA kernel: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }
    // Синхронизация устройств
    cuerr = cudaDeviceSynchronize();
    if (cuerr != cudaSuccess)
    {
        fprintf(stderr, "Cannot synchronize CUDA kernel: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }
    // Установка точки окончания
    cuerr = cudaEventRecord(stop, 0);
    if (cuerr != cudaSuccess)
    {
        fprintf(stderr, "Cannot copy c array from device to host: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }

    // Копирование результата с девайса на хост
    cuerr = cudaMemcpy(c, cdev, n2b, cudaMemcpyDeviceToHost);
    if (cuerr != cudaSuccess)
    {
        fprintf(stderr, "Cannot copy c array from device to host: %s\n",
            cudaGetErrorString(cuerr));
        return 0;
    }
    // подсчет времени
    cuerr = cudaEventElapsedTime(&gpuTime, start, stop);
    double time = gpuTime/ 1000;


    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(adev);
    cudaFree(bdev);
    cudaFree(cdev);
    return time;
}


// функция перемножения матриц на CPU
double execute_cpu(double* a, double* b, double* c, int n){
  double time = clock();
  double sum;
  for (int row=0; row<n; row++){
      for (int col=0; col<n; col++){
          sum = 0;
          for (int k=0; k<n; k++){
              sum += a[row*n+k]*b[k*n+col];
          }
          c[row*n+col] = sum;
      }
  }
  double endTime= clock()-time;
  return endTime/=CLOCKS_PER_SEC;
}


// функция проверки корректности перемножения матриц на CPU и GPU
int check_matrix(double* c_cpu, double*c_gpu, int n1, int m2, double e){
    for (int i=0;i<n1;++i){
        for(int j=0;j<m2;++j){
            // проверка на погрешность
            if(fabs(c_cpu[i*m2+j] - c_gpu[i*m2+j]) > e){
                return 0;
            }
        }
    }
    return 1;
}


// основная функция
int main(int argc, char* argv[])
{
    // размерность матриц
    int n = 2000;

    // установка числа нитей в блоке по х и у(чтобы общее число не было больше 1024)
    int xthreadsPerBlock = n < 32 ? n : 32;
    int ythreadsPerBlock = n < 32 ? n : 32;
    // установка числа блоков в гриде и нитей в блоке, чтобы одна нить обрабатывала один элемент матрицы
    dim3 blocksPerGrid =  dim3(ceil((float)n / xthreadsPerBlock), ceil((float)n / ythreadsPerBlock));
    dim3 threadsPerBlock =  dim3(xthreadsPerBlock, ythreadsPerBlock);

    // Выделение памяти на хосте
    double* a = (double*)malloc(sizeof(double)*n*n);
    double* b = (double*)malloc(sizeof(double)*n*n);
    double* c_cpu = (double*)malloc(sizeof(double)*n*n);
    double* c_gpu = (double*)malloc(sizeof(double)*n*n);
    // заполнение матриц
    for (int i = 0; i < n*n; i++) {
        a[i] = 1/1000;
        b[i] = 1/1000;
        c_cpu[i]=0;
        c_gpu[i]=0;
    }
    // вызов перемножения матриц на CPU
    double time_cpu = execute_cpu(a,b,c_cpu,n);
    printf("TIME_CPU: %f\n", time_cpu);
    // вызов перемножения матриц на GPU
    double time_gpu = execute_gpu(a,  b,  c_gpu, n, blocksPerGrid, threadsPerBlock);
    printf("TIME_GPU: %f\n", time_gpu);
    // подсчет ускорения
    printf("A: %f\n", time_cpu/time_gpu);
    // проверка корректности результатов
    int res = check_matrix(c_cpu, c_gpu, n,n, 0.000001);
    printf("CHECK: %s", res?"true":"false");
    // освобождение выделенной памяти
    free(a);
    free(b);
    free(c_gpu);
    free(c_cpu);
    return 0;
}



TIME_CPU: 86.485335
TIME_GPU: 0.418623
A: 206.594756
CHECK: true
